## 모듈 import

In [1]:
#모듈 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pycaret.regression  import *


# feature selection
from sklearn.linear_model import LinearRegression
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import cross_val_score
from tqdm import tqdm

#Scailing
from sklearn.preprocessing import PowerTransformer

#Label Encoding
from sklearn.preprocessing import LabelEncoder

In [2]:
#삡운드
import winsound as sd
# 삡싸운드
def beepsound():
    fr = 2000   # range : 37 ~ 32767
    du = 1000 # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)

## data 불러오기

In [3]:
# 데이터가 저장된 경로 설정
data_path = '../data/'

# 데이터 불러오기
train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [4]:
train

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT
0,0,1.129000e+15,5.011000e+15,패션의류,상의,3
1,1,1.135000e+15,5.011000e+15,생활/건강,반려동물,3
2,2,1.135000e+15,5.011000e+15,패션의류,기타패션의류,9
3,3,1.154500e+15,5.011000e+15,식품,농산물,10
4,4,1.165000e+15,5.011000e+15,식품,가공식품,3
...,...,...,...,...,...,...
31995,31995,5.011000e+15,2.635000e+15,식품,농산물,6
31996,31996,5.011000e+15,1.168000e+15,식품,기타식품,5
31997,31997,5.011000e+15,4.119700e+15,식품,농산물,9
31998,31998,5.011000e+15,1.132000e+15,식품,농산물,3


### 전처리

In [5]:
data = pd.concat([train,test]).reset_index(drop=True);data

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT
0,0,1.129000e+15,5.011000e+15,패션의류,상의,3.0
1,1,1.135000e+15,5.011000e+15,생활/건강,반려동물,3.0
2,2,1.135000e+15,5.011000e+15,패션의류,기타패션의류,9.0
3,3,1.154500e+15,5.011000e+15,식품,농산물,10.0
4,4,1.165000e+15,5.011000e+15,식품,가공식품,3.0
...,...,...,...,...,...,...
36635,36635,5.013000e+15,4.725000e+15,식품,농산물,NaN
36636,36636,5.013000e+15,2.826000e+15,식품,농산물,NaN
36637,36637,5.013000e+15,4.311100e+15,식품,농산물,NaN
36638,36638,5.013000e+15,4.145000e+15,식품,농산물,NaN


In [6]:
# 인코딩과 스케일링을 위함
data_index = data['index']
y_data = data['INVC_CONT']

data = data.drop(columns=['index', 'INVC_CONT'])

- 결측치 채움 -> 없음

- Encoding, Scailing

In [7]:
# # 수동으로 범주형 변수로 만들어줌
# data['SEND_SPG_INNB'] = data['SEND_SPG_INNB'].astype('str')
# data['REC_SPG_INNB'] = data['REC_SPG_INNB'].astype('str')

In [8]:
# 범주형 변수와 수치형 변수를 분리
cat_features = data.select_dtypes(include=['object']).columns.to_list()
num_features = data.select_dtypes(exclude='object').columns.to_list() 

In [9]:
#수치형변수 scailing
# data[num_features] = PowerTransformer(standardize=True).fit_transform(data[num_features])

# # #라벨인코딩
# # #위의 범주형 변수들을 라벨인코딩
# for f in cat_features:
#     data[f] = LabelEncoder().fit_transform(data[f])

# 원핫인코딩
data = pd.get_dummies(data)

In [11]:
#위에서 떼어놨던 피처 병합
data = pd.concat([data_index, data],axis=1)
data = pd.concat([data, y_data],axis=1)

- Data split

In [12]:
#train과 test를 분리
train = data.iloc[:32000,:]
test = data.iloc[32000:,:]
test = test.drop(columns = ['INVC_CONT']).reset_index(drop=True)

In [13]:
train

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT
0,0,7,249,4,12,3.0
1,1,10,249,1,11,3.0
2,2,10,249,4,6,9.0
3,3,17,249,2,8,10.0
4,4,21,249,2,0,3.0
...,...,...,...,...,...,...
31995,31995,235,33,2,8,6.0
31996,31996,235,22,2,5,5.0
31997,31997,235,86,2,8,9.0
31998,31998,235,9,2,8,3.0


### Feature selection-SelectPercentile

In [14]:
# train_index = train['index']
# test_index = test['index']
# y_train = train['INVC_CONT']
# train = train.drop(columns=['index','INVC_CONT'])
# test = test.drop(columns=['index'])

# # 사용할 모델 설정 (속도가 빠른 모델 사용 권장)
# model = LinearRegression()

# # 각 특성과 타깃(class) 사이에 유의한 통계적 관계가 있는지 계산하여 특성을 선택하는 방법 
# cv_scores = []
# for p in tqdm(range(5,100,1)):
#     X_new = SelectPercentile(percentile=p).fit_transform(train, y_train)    
#     cv_score = cross_val_score(model, X_new, y_train, scoring='rmse', cv=5).mean()
#     cv_scores.append((p,cv_score))
    
#     # Print the best percentile
# best_score = cv_scores[np.argmax([score for _, score in cv_scores])]
# print(best_score)

# # Plot the performance change with p
# plt.plot([k for k, _ in cv_scores], [score for _, score in cv_scores])
# plt.xlabel('Percent of features')
# plt.grid()

In [15]:
# # 위에서 구한 값으로 피처를 바꿔줌
# fs = SelectPercentile(percentile=best_score[0]).fit(train, y_train)
# train = fs.transform(train)
# test = fs.transform(test)

# print(train.shape)
# print(data.drop(columns=['매칭성공여부']).iloc[:,1:].columns[fs.get_support()].tolist())

## Modeling with pycaret

- Model setting

In [16]:
# #pycaret에 넣도록 변형
# train = pd.concat([pd.DataFrame(train),y_train],axis=1)
# test = pd.concat([test_index, pd.DataFrame(test)],axis=1)

In [17]:
train = train.iloc[:,1:]

In [18]:
reg = setup(train, 
            preprocess = False, # True로 설정되면, 자체적인 Feature Engineering을 추가로 진행해 Predict가 불가능해진다.
            train_size = 0.999,  # 우리는 전체 데이터를 학습해 test를 예측하는게 목표이기 때문에, 0.999로 설정한다.
            target = 'INVC_CONT', # 목표 변수는 매칭성공여부이다.
            silent = True, # 엔터를 누르기 귀찮다. 궁금하면 풀어보세요
            use_gpu = True, # GPU가 있으면 사용하세요 (Cat BOost 속도 향상)
#             numeric_features=['전체회차','현재회차'], # 숫자로써의 의미가 있다.
            session_id = 42,
            fold_shuffle = True
            )

,Description,Value
0,session_id,42
1,Target,INVC_CONT
2,Original Data,"(32000, 5)"
3,Missing Values,False
4,Numeric Features,4
5,Categorical Features,0
6,Transformed Train Set,"(31967, 4)"
7,Transformed Test Set,"(33, 4)"
8,Shuffle Train-Test,True
9,Stratify Train-Test,False


In [19]:
top5 = compare_models(sort='RMSE', fold=5, n_select = 5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,2.1770,29.2564,5.4058,0.1117,0.4156,0.4202,1.7700
gbr,Gradient Boosting Regressor,2.1739,29.4505,5.4235,0.1076,0.4107,0.4201,0.6360
catboost,CatBoost Regressor,2.1740,30.0799,5.4778,0.0867,0.4128,0.4180,3.9700
xgboost,Extreme Gradient Boosting,2.2018,30.8946,5.5525,0.0598,0.4202,0.4223,1.1200
rf,Random Forest Regressor,2.2654,31.9167,5.6429,0.0298,0.4363,0.4337,0.5620
lr,Linear Regression,2.2466,32.8435,5.7267,0.0080,0.4209,0.4365,0.0100
ridge,Ridge Regression,2.2466,32.8435,5.7267,0.0080,0.4209,0.4365,0.0080
lar,Least Angle Regression,2.2466,32.8435,5.7267,0.0080,0.4209,0.4365,0.0080
br,Bayesian Ridge,2.2463,32.8450,5.7268,0.0079,0.4208,0.4364,0.0120
lasso,Lasso Regression,2.2459,32.8458,5.7269,0.0079,0.4206,0.4364,0.0140


- Hyperparameter tuning

In [20]:
#Light Gradient Boosting Machine 모델 생성
lightgbm = create_model('lightgbm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,2.1803,35.3441,5.9451,0.0605,0.4136,0.4188
1,2.1259,25.1611,5.0161,-0.1165,0.4119,0.4180
2,2.1198,24.9103,4.9910,0.0169,0.4131,0.4184
3,2.1134,26.9672,5.1930,0.2734,0.4057,0.4173
4,2.1479,21.2927,4.6144,0.2127,0.4139,0.4105
5,2.2074,34.4452,5.8690,0.2065,0.4165,0.4219
6,2.1409,27.3968,5.2342,0.3240,0.4085,0.4217
7,2.2811,33.7329,5.8080,-0.0083,0.4363,0.4294
8,2.2423,33.3277,5.7730,-0.0435,0.4194,0.4226
9,2.1719,28.1479,5.3055,0.1231,0.4097,0.4196


### Ensemble

- blend model

In [25]:
# # 사용할 모델 수동으로 넣어줌
# models = [
#     catboost,
#     et,
#     rf
# ]

In [12]:
# final_model = blend_models(estimator_list = models, method = 'soft', optimize = 'F1')
# final_model = finalize_model(final_model) # 전체 데이터로 재학습

- Fit & predict

In [21]:
model = lightgbm
model.fit(train.iloc[:,:-1],train['INVC_CONT'])
pred = model.predict(test.iloc[:,1:])

In [22]:
submission['INVC_CONT'] = pred

### Make submission 

In [23]:
t = pd.Timestamp.now()
fname = f'submission_{t.month:02}{t.day:02}{t.hour:02}{t.minute:02}.csv'
# fname = 'submission_etpercentile.csv'
submission.to_csv('../submission/'+fname, index=False)
print("'{}' is ready to submit." .format(fname))

'submission_12062047.csv' is ready to submit.


In [24]:
beepsound()

# END